In [1]:
import numpy
import pandas
import sklearn
import sklearn.preprocessing
import sklearn.feature_extraction
import sklearn.model_selection
import sklearn.naive_bayes

# Import from CSV

In [2]:
df_raw = pandas.read_csv(
    'data.csv',
    sep=';',
    encoding='utf8',
    header=0,
    # ignore column 'Waehrung' as it is always 'EUR'
    # ignore column 'Valutadatum' as only one row varies from 'Buchungstag'
    # ignore column 'Auftragskonto' as it only has 2 distinct values and 20% of the rows have no value
    # ignore columns 'Kontonummer' and 'BLZ' to improve generalization of the model
    usecols=lambda name: name not in ['Waehrung', 'Valutadatum', 'Auftragskonto', 'Kontonummer', 'BLZ'],
    # remaining: Buchungstag, Buchungstext, Verwendungszweck, Beguenstigter/Zahlungspflichtiger, Betrag, label
    dayfirst=True,
    parse_dates=['Buchungstag'],
    index_col=0,
    converters={'Betrag': lambda s: pandas.to_numeric(s.replace(",", "."), errors='coerce')}
)
df_raw.head(3)

,Buchungstag,Buchungstext,Verwendungszweck,Beguenstigter/Zahlungspflichtiger,Betrag,label
0,2016-07-28,Lohn / Gehalt,Gehalt Adorsys GmbH & Co. KG End-To-End-Ref.: ...,Adorsys GmbH & Co. KG,2000.0,income
1,2016-07-27,Miete,Byladem1Sbt De12773501123456789889 Miete Beuth...,Georg Tasche,-670.0,living
2,2016-07-21,Bargeld,"21.07/16.34Uhr Nuernberg All Eur 70,00 Geb.Eur...",Bargeld,-70.0,private


# Test for missing values

In [3]:
df_raw.isna().sum()

Buchungstag                          0
Buchungstext                         0
Verwendungszweck                     0
Beguenstigter/Zahlungspflichtiger    0
Betrag                               0
label                                0
dtype: int64

# Preprocessing categorical data

## Buchungstext

In [4]:
buchungstext_ordinal_ndarray, buchungstext_category = df_raw['Buchungstext'].astype('category').factorize()
buchungstext_ordinal_df = pandas.DataFrame(data=buchungstext_ordinal_ndarray, columns=['Buchungstext_ordinal'])
buchungstext_ordinal_df.head(3)

,Buchungstext_ordinal
0,0
1,1
2,2


In [5]:
buchungstext_onehot_df = pandas.get_dummies(df_raw['Buchungstext'], prefix='Buchungstext_')
buchungstext_onehot_df.head(3)

,Buchungstext__Abschluss,Buchungstext__Bargeld,Buchungstext__Dauerauftrag,Buchungstext__Drogerie,Buchungstext__Euro-Überweisung,Buchungstext__Euroscheck / Gaa-Belastung Ausland,Buchungstext__Gehalt/Rente,Buchungstext__Geldanlage,Buchungstext__Lastschrift (Einzugsermächtigung),Buchungstext__Lebensmittel / Getraenke,Buchungstext__Lohn / Gehalt,Buchungstext__Miete,Buchungstext__Mieteinnahmen,Buchungstext__Spontanausgabe
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0


## label

In [6]:
df_raw['label'].value_counts()

leisure             65
standardOfLiving    47
finance             33
living              26
private             21
income              17
Name: label, dtype: int64

In [7]:
label_encoder = sklearn.preprocessing.OrdinalEncoder(dtype=numpy.int)
label_ordinal = label_encoder.fit_transform(df_raw['label'].to_numpy().reshape(-1, 1))
label_ordinal.shape

(209, 1)

In [8]:
pandas.value_counts(label_ordinal.ravel())

2    65
5    47
0    33
3    26
4    21
1    17
dtype: int64

# Feature extraction for Buchungstag

Unused, left for experiments.

In [9]:
buchungstag_df = pandas.DataFrame()
buchungstag_df['TageSeitMontag'] = df_raw['Buchungstag'].dt.dayofweek
buchungstag_df['TageSeitMonatsanfang'] = numpy.divide(df_raw['Buchungstag'].dt.day - 1,
                                                      df_raw['Buchungstag'].dt.daysinmonth - 1)
buchungstag_df.head(3)

,TageSeitMontag,TageSeitMonatsanfang
0,3,0.900000
1,2,0.866667
2,3,0.666667


# Feature extraction for Betrag

Unused, left for experiments.

In [10]:
betrag_df = pandas.DataFrame()
betrag_df['BetragPositiv'] = df_raw['Betrag'] >= 0.0
betrag_df['BetragCents00'] = numpy.round(df_raw['Betrag']) - df_raw['Betrag'] == 0.0
betrag_df['Betrag2Log10'] = numpy.round(numpy.log10(numpy.abs(df_raw['Betrag']) + 1.0))
betrag_df.head(3)

,BetragPositiv,BetragCents00,Betrag2Log10
0,True,True,3.0
1,False,True,3.0
2,False,True,2.0


# Preprocessing textual columns

In [11]:
# merge the two text columns
# delete bank account numbers (IBAN) and some other combinations of letters with digits
# ~50% less unique words without lowering classifier performance (and improves generalization)
text_series = df_raw['Verwendungszweck'].\
    str.replace(r'\bde\d+\b', '', case=False, regex=True).\
    str.cat(others=df_raw['Beguenstigter/Zahlungspflichtiger'], sep=' ').\
    str.replace(r'\b\w+\d+\w+\b', '', regex=True).\
    str.replace(r'\b\d+\w*\b', '', regex=True)
vectorizer = sklearn.feature_extraction.text.CountVectorizer(min_df=3, binary=False)
# min_df=1/2/3: overfitting for 1 and 2, optimum for 3, underfitting for 4+
# binary=True/False: practically no difference
# compared with TfidfVectorizer: superior performance
bag_of_words_matrix = vectorizer.fit_transform(text_series.to_numpy())  # sparse matrix (csr_matrix)
bag_of_words_column_names = vectorizer.get_feature_names()
bag_of_words_df = pandas.DataFrame(data=bag_of_words_matrix.todense(), columns=bag_of_words_column_names)
bag_of_words_df.shape

(209, 104)

# Create joint DataFrame

In [12]:
dataframes_to_join = [buchungstext_onehot_df, bag_of_words_df]  # left out buchungstag_df, betrag_df
X_df: pandas.DataFrame = pandas.concat(dataframes_to_join, axis=1)
X_column_names = []
for df in dataframes_to_join:
    X_column_names.extend(df.columns)
X_df.columns = X_column_names
X_df.head(3)

,Buchungstext__Abschluss,Buchungstext__Bargeld,Buchungstext__Dauerauftrag,Buchungstext__Drogerie,Buchungstext__Euro-Überweisung,Buchungstext__Euroscheck / Gaa-Belastung Ausland,Buchungstext__Gehalt/Rente,Buchungstext__Geldanlage,Buchungstext__Lastschrift (Einzugsermächtigung),Buchungstext__Lebensmittel / Getraenke,...,vag,verf,verfa,verfal,verfall,verfalld,vj,vk,wie,zalando
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
y = label_ordinal.ravel()  # 2D -> 1D
y.shape

(209,)

# Train classifier

The performance metrics for a single trained model are not very robust because the test set is quite small. (The performance of a trained model is also not very robust because of the small training set. This means a model trained from another random training set can perform quite differently.)

Crossvalidation is used to determine mean and standard deviation for the performance metrics over multiple trained models. Stratified crossvalidation ensures that the distribution of classes among samples in training and test sets is not skewed. Repeated crossvalidation (with randomly selected samples each time) lowers the variance further. However, classes with only a few samples put a lower bound on the variance.

In [14]:
n_splits = 3  # test set made up of 33% of the data, rest for training set
n_repeats = 40  # metrics results stabilize for 40+ repeats
crossval = sklearn.model_selection.RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats)
classifier_mnb = sklearn.naive_bayes.MultinomialNB(alpha=1.0)  # best performance for alpha=1.0
# sklearn.naive_bayes.ComplementNB performs slightly worse than MultinomialNB
metric_names = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
scores = sklearn.model_selection.cross_validate(classifier_mnb, X_df, y=y, cv=crossval,
                                                return_train_score=True, scoring=metric_names)

# Evaluate classifier

In [15]:
index_set_names = ['testing', 'training']
eval_df = pandas.DataFrame(
    index=pandas.MultiIndex(
        names=['metric', 'over set'],
        levels=[metric_names, index_set_names],
        codes=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]]
    ),
    columns=['mean', 'stddev'],
    dtype=numpy.float64
)
for metric_name in metric_names:
    for index_set_name in index_set_names:
        score_key = index_set_name.rsplit('ing')[0] + '_' + metric_name
        metric_data = scores[score_key]
        eval_df.at[(metric_name, index_set_name), 'mean'] = metric_data.mean()
        eval_df.at[(metric_name, index_set_name), 'stddev'] = metric_data.std()
eval_df

mean    stddev
metric          over set                    
accuracy        testing   0.891449  0.034500
                training  0.911963  0.014135
precision_macro testing   0.928857  0.023004
                training  0.939484  0.009963
recall_macro    testing   0.885883  0.038592
                training  0.904238  0.016935
f1_macro        testing   0.898182  0.033469
                training  0.916584  0.014350